In [1]:
import numpy as np
import pandas as pd
import random
import nengo

from scipy.special import legendre
import scipy.linalg

# Create Random Data
Replace every value in the high engagement dataset with a random value

In [2]:
high = np.load('higheng.npy',allow_pickle=True)
high = high[:,1]

In [3]:
np.random.seed()
for i in range(len(high)):
    for j in range(len(high[i])):
        for k in range(len(high[i][j])):
            high[i][j][k]=np.random.rand()
np.save('rand_data.npy', high)

# Create LMU Pre-processed Datasets

Aaron's code

In [4]:
# Define a nengo.Process that implements an LMU.  This can be placed inside a
#  nengo.Node
class Legendre(nengo.Process):
    def __init__(self, theta, q, size_in=1):
        self.q = q              # number of internal state dimensions per input
        self.theta = theta      # size of time window (in seconds)
        self.size_in = size_in  # number of inputs
        
        # Do Aaron's math to generate the matrices
        #  https://github.com/arvoelke/nengolib/blob/master/nengolib/synapses/analog.py#L536
        Q = np.arange(q, dtype=np.float64)
        R = (2*Q + 1)[:, None] / theta
        j, i = np.meshgrid(Q, Q)
    
        self.A = np.where(i < j, -1, (-1.)**(i-j+1)) * R
        self.B = (-1.)**Q[:, None] * R

        super().__init__(default_size_in=size_in, default_size_out=q*size_in)
        
    def make_step(self, shape_in, shape_out, dt, rng, state=None):
        state=np.zeros((self.q, self.size_in))
        
        # Handle the fact that we're discretizing the time step
        #  see https://en.wikipedia.org/wiki/Discretization#Discretization_of_linear_state_space_models
        Ad = scipy.linalg.expm(self.A*dt)
        Bd = np.dot(np.dot(np.linalg.inv(self.A), (Ad-np.eye(self.q))), self.B)
        
        # this code will be called every timestep
        def step_legendre(t, x, state=state):
            state[:] = np.dot(Ad, state) + np.dot(Bd, x[None,:])
            return state.T.flatten()
        return step_legendre

    def get_weights_for_delays(self, r):
        # compute the weights needed to extract the value at time r
        # from the network (r=0 is right now, r=1 is theta seconds ago)
        r = np.asarray(r)
        m = np.asarray([legendre(i)(2*r - 1) for i in range(self.q)])
        return m.reshape(self.q,-1).T

Load raw data

In [5]:
high = np.load('higheng.npy',allow_pickle=True)
high = high[:,1]

mid = np.load('mideng.npy',allow_pickle=True)
mid = mid[:,1]

low = np.load('loweng.npy',allow_pickle=True)
low = low[:,1]

random = np.load('rand_data.npy',allow_pickle=True)

Define parameters. <br>
theta = length of time (seconds) 
q = convert every 1 value into q new values which characterises the input over the last theta seconds 

In [6]:
theta = 3
q = 4
dt = 0.001

Normalize the data by subtracting clip mean and dividing by clip sd for each clip.

In [7]:
h=np.vstack(high)
m=np.vstack(mid)
l=np.vstack(low)
r=np.vstack(random)

#calculate mean and sd for high and low engagement combined
hl_mean = np.vstack(np.concatenate((h,l))).mean(axis=0)
hl_sd = np.vstack(np.concatenate((h,l))).std(axis=0)

#calculate mean and sd for intermediate engagement separately
m_mean = m.mean(axis=0) 
m_sd = m.std(axis=0) 

#calculate mean and sd for random data separately
r_mean = r.mean(axis=0) 
r_sd = r.std(axis=0) 

In [8]:
#normalize high engagement
for h in high:
    h -= hl_mean
    h /= hl_sd

#normalize low engagement
for l in low:
    l -= hl_mean
    l /= hl_sd

#normalize intermediate engagement
for m in mid:
    m -= m_mean
    m /= m_sd
    
#normalize random data
for r in random:
    r -= r_mean
    r /= r_sd

Implement LMU on normalized data

In [9]:
high_lmu=[]

for i in high:
    lmu = Legendre(theta=theta, q=q, size_in=184)
    y = lmu.apply(i, dt=dt) 
    decoder = lmu.get_weights_for_delays(0)
    high_lmu.append(y)
    
high_lmu=np.asarray(high_lmu)  

In [10]:
mid_lmu=[]

for i in mid:
    lmu = Legendre(theta=theta, q=q, size_in=184)
    y = lmu.apply(i, dt=dt) 
    decoder = lmu.get_weights_for_delays(0)
    mid_lmu.append(y)
    
mid_lmu=np.asarray(mid_lmu)  

In [11]:
low_lmu=[]

for i in low:
    lmu = Legendre(theta=theta, q=q, size_in=184)
    y = lmu.apply(i, dt=dt) 
    decoder = lmu.get_weights_for_delays(0)
    low_lmu.append(y)
    
low_lmu=np.asarray(low_lmu)  

In [12]:
rand_lmu=[]

for i in random:
    lmu = Legendre(theta=theta, q=q, size_in=184)
    y = lmu.apply(i, dt=dt) 
    decoder = lmu.get_weights_for_delays(0)
    rand_lmu.append(y)
    
rand_lmu=np.asarray(rand_lmu)  

In [13]:
np.save('high_lmu'+str(q)+'_theta'+str(theta)+'.npy', high_lmu, allow_pickle=True)
np.save('mid_lmu'+str(q)+'_theta'+str(theta)+'.npy', mid_lmu, allow_pickle=True)
np.save('low_lmu'+str(q)+'_theta'+str(theta)+'.npy', low_lmu, allow_pickle=True)
np.save('rand_lmu'+str(q)+'_theta'+str(theta)+'.npy', rand_lmu, allow_pickle=True)